In [7]:
import pyspark as ps
from pyspark.ml import Pipeline

## Read data.json into Spark SQL context

In [5]:
data_file = 'data/data.json'
df = spark.read.json(data_file)

In [6]:
print(df.printSchema())
print(df.count())
df.show(3)

root
 |-- author: string (nullable = true)
 |-- excerpt: string (nullable = true)
 |-- title: string (nullable = true)

None
9050
+----------+--------------------+-----------------+
|    author|             excerpt|            title|
+----------+--------------------+-----------------+
|JaneAusten|Chapter 1 || It i...|PrideAndPrejudice|
|JaneAusten|“What is his name...|PrideAndPrejudice|
|JaneAusten|“In such cases, a...|PrideAndPrejudice|
+----------+--------------------+-----------------+
only showing top 3 rows



## 

In [8]:
tokenizer = 0
pipeline = Pipeline(stages=[tokenizer])

NameError: name 'tokenizer' is not defined